In [7]:
using Newtonsoft.Json;
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;

In [2]:
public class Entry
{
    public string name { get; set; }
    public string group { get; set; }
    public string discipline { get; set; }
    public int mark { get; set; }
}

public class Input
{
    public string taskName { get; set; }
    public Entry[] data { get; set; }
}

public class Output
{
    public List<Dictionary<string, object>> Response { get; set; }
    public Output(List<Dictionary<string, object>> Response)
    {
        this.Response = Response;
    }
}

In [6]:
public Output GetStudentsWithHighestGPA(Entry[] input)
{
    List<Dictionary<string, object>> output = new List<Dictionary<string, object>>();
    var studentToGpa = input
            .GroupBy(e => e.name)
            .Select(g => new { name = g.Key, GPA = g.Average(e => e.mark) });
    studentToGpa
            .Where(e => e.GPA == studentToGpa.Max(c => c.GPA))
            .Select(e => e.name)
            .ToList()
            .ForEach(e => output.Add(new Dictionary<string, object>() {{"Cadet", e}, {"GPA", studentToGpa.Max(c => c.GPA)}}));

    return new Output(output);
}

public Output CalculateGPAByDiscipline(Entry[] input)
{
    List<Dictionary<string, object>> output = new List<Dictionary<string, object>>();
    input
        .GroupBy(e => e.discipline)
        .Select(g => new { discipline = g.Key, GPA = g.Average(s => s.mark) })
        .ToList()
        .ForEach(e => output.Add(new Dictionary<string, object>() { {e.discipline, e.GPA} } ));
    
    return new Output(output);
}

public Output GetBestGroupsByDiscipline(Entry[] input)
{
    List<Dictionary<string, object>> output = new List<Dictionary<string, object>>();
    input
        .GroupBy(e => e.discipline)
        .Select(g => new { discipline = g.Key, groups = g.GroupBy(s => s.group)
        .Select(g => new { group = g.Key, GPA = g.Average(s => s.mark) }) })
        .Select(g => (Discipline: g.discipline, Group: g.groups.OrderByDescending(g => g.GPA).First().group, GPA: g.groups.OrderByDescending(g => g.GPA).First().GPA))
        .ToList()
        .ForEach(e => output.Add(new Dictionary<string, object>() { {"Discipline", e.Discipline }, {"Group", e.Group }, {"GPA", e.GPA } }));

    return new Output(output);
}

In [11]:
string[] files = { "GetStudentsWithHighestGPA.json", "CalculateGPAByDiscipline.json", "GetBestGroupsByDiscipline.json" };
foreach (string name in files)
{
    var file = JsonConvert.DeserializeObject<Input>(File.ReadAllText(name));
    switch (file.taskName)
    {
        case "GetStudentsWithHighestGPA":
            File.WriteAllText("OutGetStudentsWithHighestGPA.json", JsonConvert.SerializeObject(GetStudentsWithHighestGPA(file.data)));
            break;
        case "CalculateGPAByDiscipline":
            File.WriteAllText("OutCalculateGPAByDiscipline.json", JsonConvert.SerializeObject(CalculateGPAByDiscipline(file.data)));
            break;
        case "GetBestGroupsByDiscipline":
            File.WriteAllText("OutGetBestGroupsByDiscipline.json", JsonConvert.SerializeObject(GetBestGroupsByDiscipline(file.data)));
            break;
    }
}